# Copilot X Glint Sentiment Relationship Strength
This Jupyter Notebook analyzes employee engagement data, focusing on "Copilot Usage" metrics. 
The code performs the following tasks:
1. Imports and cleans data from a CSV file that includes both Glint Score and Copilot Usage by user (Viva Insights Export).
2. Converts engagement scores to a 100-point scale and categorizes them into favorability groups (droping NEU responses).
3. Calculates weekly Copilot usage for employees.
4. Categorizes employees into user groups (e.g., PowerUser, NoviceUser) based on their Copilot activity.
5. Analyzes the relationship between Copilot usage categories and engagement scores using odds ratios.
6. Outputs insights on the strongest relationships between user categories and engagement metrics.


### Install Libraries and initialization 


In [1]:
# pip install --user pandas
#pip install --user scipy
# file_name: Path to the CSV file containing the data.
# item_options: Number of options in the survey (default is 5 for a 5-point scale).
# survey_close_date: The date when the survey closed (format: YYYY-MM-DD).

import chardet
import re
from scipy.stats import percentileofscore
import datetime
import pandas as pd



survey_close_date = datetime.datetime.strptime('2024-09-29', '%Y-%m-%d')
file_name = 'C:/Users/bentankus/OneDrive - Microsoft/Projects/Copilot Support/glint_demodata_singlesource.csv'
item_options = 5 # Number of options in the survey (default is 5 for a 5-point scale).


######## IMPORT DATA  ########
# The code will consider any data with "Copilot" in the column name as copilot activities.

# EXPECTED DATA STRUCTURE
## | EMPLOYEE IDENTIFIER | GLINT ITEM | GLINT ITEM SCORE | COPILOT ITEMS ... | DATE OF COPILOT WEEKLY USAGE | 

# SINGLE DATA SOURCE
# SENSE DATA ENCODING
with open(file_name, "rb") as f:
    result = chardet.detect(f.read())
    print(f"Detected Encoding: {result['encoding']}")
# Load the Excel file into a pandas DataFrame
df_glint = pd.read_csv(file_name , encoding=result['encoding'])


Detected Encoding: ascii


In [2]:
####### CLEAN DATA  ########
# (Convert to favorability and drop neutral responses)

# CONVERT TO 100PT SCALE FOR EASY FAVORABILITY PARSING
df_glint['score100'] = (df_glint['Score'] - 1) * (100/(item_options-1))

# CONVERT TO FAVORABILITY
df_glint['favorability'] = df_glint['score100'].apply(lambda x: 'fav' if x > 70 else ('unfav' if x < 40 else 'neu'))

# CREATE FULL DF FOR LATER USE
df_glint_full = df_glint

# DROP NEUTRAL FAVORABILITY
df_glint = df_glint[df_glint['favorability'] != 'neu']


### Analyse Data

Calculate "Copilot Usage" based on 12 week habit building literature and Odds Ratio methods comparing differing user types

In [3]:
#Calculate "Copilot Usage" based on 12 week habit building literature and Odds Ratio methods comparing differing user types


# CALCULATE COPILOT USAGE
columns_to_include = ['Employee_ID', 'MetricDate']
# Dynamically filter columns based on conditions
copilot_columns = df_glint.filter(
    regex='(?i)copilot', axis=1
).columns

# Additional filtering logic can be added here
# Example: Exclude columns containing specific keywords
exclude_keywords = ['Test', 'day']
copilot_columns = copilot_columns[~copilot_columns.str.contains('|'.join(exclude_keywords), case=False)]

# Select employee_id, date, and copilot-related columns
df_glintscore_copilot = df_glint[columns_to_include + copilot_columns.tolist()].drop_duplicates()
df_glintscore_copilot['MetricDate'] = pd.to_datetime(df_glintscore_copilot['MetricDate'], format='%m/%d/%Y', errors='coerce')

# Define the start date for the past 12 weeks based on the survey close date
start_date = survey_close_date - datetime.timedelta(weeks=12)

# Filter data for the past 12 weeks
df_past_12_weeks = df_glintscore_copilot[
    (df_glintscore_copilot['MetricDate'] >= start_date) & 
    (df_glintscore_copilot['MetricDate'] < survey_close_date)
]

# COUNT ALL NON-ZERO COPILOT ACTIVITIES FOR EACH EMPLOYEE_ID AND DATE
df_past_12_weeks['NonZero_Copilot_Activities'] = df_past_12_weeks.iloc[:, 2:].apply(lambda row: (row > 0).sum(), axis=1)

# Group by Employee_ID and calculate weekly usage
weekly_usage = df_past_12_weeks.groupby('Employee_ID')['NonZero_Copilot_Activities'].agg(
    total_usage='sum',
    avg_usage='mean'
).reset_index()


# JOIN IN df_past_12_weeks_usage NonZero_Copilot_Activities FOR EACH USER TO weekly_usage SO WE HAVE HABIT BUILDING
weekly_usage = weekly_usage.merge(df_past_12_weeks[['Employee_ID', 'NonZero_Copilot_Activities']].drop_duplicates(), on='Employee_ID', how='left')



C:\Users\bentankus\AppData\Local\Temp\ipykernel_9128\1602427854.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_past_12_weeks['NonZero_Copilot_Activities'] = df_past_12_weeks.iloc[:, 2:].apply(lambda row: (row > 0).sum(), axis=1)


### Assign user categories

In [6]:
# Assign user categories
# Power User: averaging 15+ weekly total Copilot actions and any use of Copilot in at least 9 out of past 12 weeks.
# Habitual User: any use of Copilot in at least 9 out of past 12 weeks.
# Novice User: averaging at least one Copilot action over the last 12 weeks.
# Low User: having any Copilot action in the past 12 weeks.
# Non-user: zero Copilot actions in the last 12 weeks.
def assign_user_category(row):
    if row['total_usage'] > 0:
        if row['NonZero_Copilot_Activities'] >= 9:
            if row['avg_usage'] >= 15:
                return 'PowerUser'
            else:
                return 'HabitualUser'
    
        elif row['avg_usage'] >= 1:
            return 'NoviceUser'
        
        elif row['total_usage'] == 1:
            return 'LowUser'

        else:
            return 'NonUser'
    
    return 

weekly_usage['UserCategory'] = weekly_usage.apply(assign_user_category, axis=1)

# RENAME COLUMN
weekly_usage['UserCategory_Group'] = weekly_usage['UserCategory']

# Merge the user categories back into the original DataFrame
df_past_12_weeks_usage = df_past_12_weeks.merge(
    weekly_usage[['Employee_ID', 'UserCategory_Group']],
    on='Employee_ID',
    how='left'
)

# MERGE USERCATEGORY BACK INTO GLINT DATA
df_glint_usage = df_glint.merge(
    df_past_12_weeks_usage[['Employee_ID', 'UserCategory_Group']],
    on='Employee_ID',
    how='left'
) [['Employee_ID', 'Glint_Item', 'Score', 'UserCategory_Group']]


# OUTPUT THE COUNT OF EACH USER CATEGORY AND PERCENT OF TOTAL FORMATTED IN A TABLE
user_category_counts = weekly_usage['UserCategory_Group'].value_counts()
user_category_percent = user_category_counts / user_category_counts.sum() * 100
user_category_table = pd.concat([user_category_counts, user_category_percent], axis=1)
user_category_table.columns = ['Count', 'Percent']
user_category_table = user_category_table.round(2)
print(user_category_table)


# ADD A USER CATEGORY FLAG ALERT THAT READS user_category_table AND PRINTS ALERT IF ANY PERCENT IS LESS THAN 5%
def check_user_category_alerts(user_category_table):
    for category, row in user_category_table.iterrows():
        if row['Percent'] < 5:
            print('')
            print(f"WARNING: {category} is below 5% with {row['Count']} users ({row['Percent']:.2f}%)")
            print("Consider redefining user categories to ensure each group has a minimum of 5% of the total user base.")

check_user_category_alerts(user_category_table)



                    Count  Percent
UserCategory_Group                
NoviceUser            272    98.55
HabitualUser            4     1.45

Consider redefining user categories to ensure each group has a minimum of 5% of the total user base.


### Convert to Favorability and begin the Odds Ratio calculation

In [7]:
# ADD FAVORABILITY COLUMN
df_glint_usage['favorability'] = df_glint_usage['Score'].apply(lambda x: 'fav' if x >= 4 else ('unfav' if x <= 2 else 'neu'))
df_glint_usage = df_glint_usage[df_glint_usage['favorability'] != 'neu']

# GET COUNTS FOR EACH USER CATEGORY GROUPED BY GLINT ITEM AND FAVORABILITY
df_glint_usage_grouped = df_glint_usage.groupby(['Glint_Item', 'favorability', 'UserCategory_Group'])['Employee_ID'].nunique().reset_index(name='user_count')

# ADD 0.5 TO EACH COUNT TO AVOID DIVISION BY ZERO AND INF ODDS
df_glint_usage_grouped['user_count'] = df_glint_usage_grouped['user_count'] + 0.5

# PIVOT THE DATAFRAME TO GET FAV / UNFAV IN COLUMNS
df_glint_usage_grouped_pivot = df_glint_usage_grouped.pivot_table(index=['Glint_Item', 'UserCategory_Group'], columns='favorability', values='user_count').reset_index()

# DIVIDE FAV BY UNFAV TO GET ODDS
df_glint_usage_grouped_pivot['odds'] = df_glint_usage_grouped_pivot['fav'] / df_glint_usage_grouped_pivot['unfav']

# DROP NAN ODDS, THEN ADD "MAX" OR "MIN" LABEL TO THE THE MAX-MIN ODDS FOR EACH GLINT ITEM
df_glint_usage_grouped_pivot_odds = df_glint_usage_grouped_pivot.dropna(subset=['odds'])

# DROP FAV AND UNFAV COLUMN AND PIVOT OUT USERCATEGORY_GROUP VALUES INTO NEW COLUMNS
df_glint_usage_grouped_pivot_odds = df_glint_usage_grouped_pivot_odds.drop(columns=['fav', 'unfav'])
df_glint_usage_grouped_pivot_odds = df_glint_usage_grouped_pivot_odds.pivot(index='Glint_Item', columns='UserCategory_Group', values='odds').reset_index()







### Calculate and display final odds ratio


In [8]:
# DROP NAN ODDS, THEN ADD "MAX" OR "MIN" LABEL TO THE THE MAX-MIN ODDS FOR EACH GLINT ITEM
df_glint_usage_grouped_pivot_odds = df_glint_usage_grouped_pivot.dropna(subset=['odds'])

# DROP FAV AND UNFAV COLUMNS 
df_glint_usage_grouped_pivot_odds = df_glint_usage_grouped_pivot_odds.drop(columns=['fav', 'unfav'])

# Identify the MAX and MIN odds for each Glint_Item and UserCategory_Group
df_glint_usage_grouped_pivot_odds['MAX_MIN'] = df_glint_usage_grouped_pivot_odds.groupby('Glint_Item')['odds'].transform(
    lambda x: ['MAX' if val == x.max() else 'MIN' if val == x.min() else None for val in x])

# DROP NONE VALUES FROM MAX_MIN COLUMN
df_glint_usage_grouped_pivot_odds = df_glint_usage_grouped_pivot_odds.dropna(subset=['MAX_MIN'])

# PREPARE df_max_min FOR LATER JOIN
df_max_min = df_glint_usage_grouped_pivot_odds[['Glint_Item', 'UserCategory_Group','MAX_MIN']]

# DROP USERCATEGORY_GROUP COLUMN AND PIVOT MAX_MIN INTO COLUMNS
df_glint_usage_grouped_pivot_odds = df_glint_usage_grouped_pivot_odds.drop(columns='UserCategory_Group').drop_duplicates()
df_glint_usage_grouped_pivot_odds = df_glint_usage_grouped_pivot_odds.pivot(index='Glint_Item', columns='MAX_MIN', values='odds').reset_index()

# TRY TO DIVIDE THE MAX ODDS BY THE MIN ODDS FOR EACH GLINT ITEM. IF THERE IS NO MIN COLUMN, PRINT ('NOT ENOUGH DATA')
try:
    df_glint_usage_grouped_pivot_odds['odds_ratio'] = df_glint_usage_grouped_pivot_odds['MAX'] / df_glint_usage_grouped_pivot_odds['MIN']
except KeyError:
    print('NOT ENOUGH DATA')
    df_glint_usage_grouped_pivot_odds['odds_ratio'] = None

# DROP ANY ROW FROM df_glint_usage_grouped_pivot_odds THAT HAS A NAN VALUE
df_glint_usage_grouped_pivot_odds = df_glint_usage_grouped_pivot_odds.dropna()

# JOIN df_glint_usage_grouped_pivot_odds WITH df_max_min TO GET THE USERCATEGORY_GROUP FOR THE MAX ODDS
df_glint_usage_grouped_pivot_odds = df_glint_usage_grouped_pivot_odds.merge(df_max_min, on='Glint_Item', how='left')

# Group by 'Glint_Item' and 'odds_ratio', then concatenate 'UserCategory_Group' values into a single string
df_glint_usage_grouped_pivot_odds_final = df_glint_usage_grouped_pivot_odds.groupby(['Glint_Item', 'odds_ratio']).agg({
    'UserCategory_Group': lambda x: '-'.join(x).replace(' ', '')
}).reset_index()





# PRINT FINAL ODDS RATIO TABLE SORTED BY ODDS RATIO
df_glint_usage_grouped_pivot_odds_final = df_glint_usage_grouped_pivot_odds_final.sort_values('odds_ratio', ascending=False).reset_index(drop=True)




# NEW LINE FOR READABILITY
print('')

print(df_glint_usage_grouped_pivot_odds_final)


     Glint_Item  odds_ratio       UserCategory_Group
0  Q_ESAT_HAPPY    1.320000  HabitualUser-NoviceUser
1  Q_JOB_GROWTH    1.248062  HabitualUser-NoviceUser


### Calculate Glint Score for each user group and item

In [9]:
df_glint_agg = df_glint_full[['Employee_ID','Glint_Item','score100']].drop_duplicates()


# JOIN IN USERCATEGORY GROUP TO df_glint_agg
df_glint_agg = df_glint_agg.merge(
    df_glint_usage[['Employee_ID', 'UserCategory_Group']].drop_duplicates(),
    on='Employee_ID',
    how='left'
)

# GROUPED BY GLINT ITEM AND USERCATEGORY GROUP AND CALCULATE THE MEAN SCORE FROM SCORE100
df_glint_agg = round(df_glint_agg.groupby(['Glint_Item', 'UserCategory_Group'])['score100'].mean().reset_index(),1)

df_glint_agg

Glint_Item UserCategory_Group  score100
0            Q_BELONGING       HabitualUser      56.2
1            Q_BELONGING         NoviceUser      54.5
2  Q_CULTURE_RECOGNITION       HabitualUser      75.0
3  Q_CULTURE_RECOGNITION         NoviceUser      56.4
4           Q_ESAT_HAPPY       HabitualUser      50.0
5           Q_ESAT_HAPPY         NoviceUser      56.1
6       Q_ESAT_RECOMMEND       HabitualUser      81.2
7       Q_ESAT_RECOMMEND         NoviceUser      50.7
8           Q_JOB_GROWTH       HabitualUser      68.8
9           Q_JOB_GROWTH         NoviceUser      57.7